<a href="https://colab.research.google.com/github/zakariazemmahi/waste-detection-yolov8/blob/main/Models/Application_de_comptur_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **⚙️ Étape 1 – Installer les dépendances**

In [2]:
!pip install streamlit ultralytics
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

# **📝 Étape 2 – Créer ton fichier app.py Streamlit**

In [15]:
import streamlit as st
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import requests
import streamlit.components.v1 as components
import cv2
from io import BytesIO
import base64
import plotly.express as px
import gc

# Configuration de la page
st.set_page_config(
    page_title="SmartWasteDetection",
    page_icon="♻️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS personnalisé
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(90deg, #4CAF50, #81C784);
        padding: 2rem;
        border-radius: 10px;
        text-align: center;
        margin-bottom: 2rem;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .main-header h1 {
        color: white;
        font-size: 3rem;
        margin: 0;
    }
    .main-header p {
        color: white;
        font-size: 1.2rem;
        margin-top: 0.5rem;
    }
    .detection-card, .stat-item {
        background: white;
        padding: 1.5rem;
        border-radius: 10px;
        box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
    }
</style>
""", unsafe_allow_html=True)

@st.cache_resource
def load_models():
    try:
        model_detect = YOLO("/content/drive/MyDrive/yolov8_best_smartdetection.pt")
        model_classify = YOLO("/content/drive/MyDrive/yolov8_best.pt")
        return model_detect, model_classify
    except Exception as e:
        st.error(f"Erreur lors du chargement des modèles : {str(e)}")
        return None, None

def process_image_detection(image, model_detect):
    img_array = np.array(image)
    results = model_detect.predict(img_array, conf=0.25, verbose=False)
    waste_objects, non_waste_objects = [], []
    if results:
        for box in results[0].boxes:
            cls_id = int(box.cls)
            class_name = results[0].names[cls_id]
            conf = float(box.conf)
            bbox = box.xyxy.cpu().numpy().astype(int)[0]
            obj = {'class': class_name, 'confidence': conf, 'bbox': bbox}
            if class_name.lower() == "dechet":
                waste_objects.append(obj)
            else:
                non_waste_objects.append(obj)
    return waste_objects, non_waste_objects, results[0]

def classify_waste(image, bbox, model_classify):
    x1, y1, x2, y2 = bbox
    cropped = np.array(image)[y1:y2, x1:x2]
    if cropped.size == 0:
        return None, 0
    results = model_classify.predict(cropped, conf=0.25, verbose=False)
    if results and results[0].boxes:
        box = results[0].boxes[0]
        return results[0].names[int(box.cls)], float(box.conf)
    return None, 0

def create_annotated_image(image, waste_objects, non_waste_objects, classifications):
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
    except:
        font = ImageFont.load_default()

    for i, obj in enumerate(waste_objects):
        x1, y1, x2, y2 = obj['bbox']
        draw.rectangle([x1, y1, x2, y2], outline="red", width=4)
        label = classifications[i][0] if classifications[i] else "Dechet"
        draw.text((x1, y1), f"{label} ({obj['confidence']:.0%})", fill="white", font=font)

    for obj in non_waste_objects:
        x1, y1, x2, y2 = obj['bbox']
        draw.rectangle([x1, y1, x2, y2], outline="green", width=4)
        draw.text((x1, y1), f"{obj['class']} ({obj['confidence']:.0%})", fill="white", font=font)

    return image

# Sidebar
with st.sidebar:
    st.image("https://cdn-icons-png.flaticon.com/512/10067/10067108.png", width=80)
    st.title("Smart Waste Detection")
    st.markdown("Projet IA ENSAM Meknès - 2025")

# Header
st.markdown("""
<div class="main-header">
    <h1>Smart Waste Detection</h1>
    <p>Analyse intelligente des déchets via YOLOv8</p>
</div>
""", unsafe_allow_html=True)

# Chargement des modèles
with st.spinner("Chargement des modèles..."):
    model_detect, model_classify = load_models()
if model_detect is None or model_classify is None:
    st.stop()

# Upload images
uploaded_files = st.file_uploader("Uploader des images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if uploaded_files:
    total_detected, total_waste, all_classes = 0, 0, []
    for uploaded_file in uploaded_files:
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption=f"Image originale: {uploaded_file.name}", use_column_width=True)

        with st.spinner("Analyse en cours..."):
            waste_objs, non_waste_objs, result = process_image_detection(image, model_detect)
            classifications = [classify_waste(image, obj['bbox'], model_classify) for obj in waste_objs]
            annotated = create_annotated_image(image.copy(), waste_objs, non_waste_objs, classifications)

        st.image(annotated, caption="Image annotée", use_column_width=True)
        total_detected += len(waste_objs) + len(non_waste_objs)
        total_waste += len(waste_objs)
        all_classes.extend([c[0] for c in classifications if c[0]])

        # Download button
        buf = BytesIO()
        annotated.save(buf, format="PNG")
        st.download_button("Télécharger l'image annotée", buf.getvalue(), file_name="annotated.png", mime="image/png")

    st.markdown("---")
    st.subheader("Statistiques globales")
    col1, col2, col3 = st.columns(3)
    col1.metric("Images traitées", len(uploaded_files))
    col2.metric("Objets détectés", total_detected)
    col3.metric("Déchets trouvés", total_waste)

    if all_classes:
        fig = px.histogram(x=all_classes, title="Types de déchets détectés", labels={'x': 'Type'})
        st.plotly_chart(fig)
else:
    st.info("Importez des images pour commencer l'analyse.")


2025-06-12 15:37:32.095 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:37:32.098 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:37:32.193 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-12 15:37:32.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:37:32.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:37:32.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 15:37:32.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

# **🌐 Étape 3 – Obtenir ton IP publique (optionnel)**

In [13]:
!wget -q -O - ipv4.icanhazip.com


34.125.255.86


# **🚀 Étape 4 – Lancer l'application et ouvrir un tunnel**

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.255.86:8501

⠴⠦your url is: https://fresh-glasses-melt.loca.lt
2025-06-12 15:38:02.902 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_module_paths
    potential_paths = extract_paths(module)
                      ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 210, in <lambda>
    lambda m: list(m.__p